# 10. 데이터 정리하기

지금까지 API와 크롤링을 통해 여러 금융 데이터를 수집하는 방법을 배웠습니다. 이번 장에서는 각각 csv 파일로 저장된 주가, 재무제표, 가치지표 데이터들을 하나로 합친 후 저장하는 과정을 살펴보겠습니다.

## 10.1 주가 정리하기

주가는 data/KOR_price 폴더 내에 티커_price.csv 파일로 저장되어 있습니다. 해당 파일들을 불러온 후 데이터를 묶는 작업을 통해 하나의 파일로 합치는 방법을 알아보겠습니다.

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

KOR_ticker = pd.read_csv('data/KOR_ticker.csv', index_col=0)
KOR_ticker['종목코드'] = KOR_ticker['종목코드'].astype(np.str).str.zfill(6)

price_list = {}

for i in tqdm(range(0, len(KOR_ticker))):
    name = KOR_ticker['종목코드'][i]
    price_list[name] = pd.read_csv('data/KOR_price/'+name+'_price.csv', index_col=0)
    
price_list = pd.DataFrame({tic: data['Close'] for tic, data in price_list.items()})
price_list = price_list.fillna(method='ffill')


100%|█████████████████████████████████████████████████████████████████████████████| 2196/2196 [00:08<00:00, 267.97it/s]


In [3]:
price_list.head()

,005930,000660,035420,051910,207940,035720,005380,006400,068270,000270,...,038160,033600,195440,058420,121890,114570,080440,347140,158310,028040
2016-04-14,26000,27250,134791,338500,NaN,100300,152000,104000,102783,48050,...,NaN,1910,NaN,30125,7812,66222,8153,NaN,10800,4485.0
2016-04-15,26000,27650,134791,335000,NaN,101200,151500,106500,101102,48150,...,NaN,1930,NaN,29091,7841,67052,8153,NaN,10000,4590.0
2016-04-18,25980,27200,132388,337000,NaN,102200,154000,107500,98018,49050,...,NaN,1865,NaN,33419,7694,68579,8153,NaN,9530,4695.0
2016-04-19,25760,27450,133990,326500,NaN,102500,153000,106500,99046,48950,...,NaN,1900,NaN,38296,7664,68108,8153,NaN,10500,5630.0
2016-04-20,25980,27550,130986,322000,NaN,103100,151500,107000,96710,48350,...,NaN,1905,NaN,38643,7694,69050,8153,NaN,11050,5250.0


In [4]:
price_list.tail()

,005930,000660,035420,051910,207940,035720,005380,006400,068270,000270,...,038160,033600,195440,058420,121890,114570,080440,347140,158310,028040
2021-04-08,84700,144000,381500,810000,763000.0,548000,231500,657000,311500,87100,...,862.0,179,2075.0,2160,739,620,498,2080.0,505,23.0
2021-04-09,83600,140000,383500,812000,770000.0,558000,228500,663000,309500,84600,...,862.0,179,2075.0,2160,739,620,498,2080.0,505,17.0
2021-04-12,83200,137500,385500,817000,782000.0,558000,226000,655000,310500,84300,...,862.0,179,2075.0,2160,739,620,498,2080.0,505,6.0
2021-04-13,84000,139500,388500,868000,784000.0,558000,230000,690000,316000,85500,...,862.0,179,2075.0,2160,739,620,498,2080.0,505,6.0
2021-04-14,83600,137000,393500,891000,789000.0,558000,234000,695000,313500,86700,...,862.0,179,2075.0,2160,739,620,498,2080.0,505,6.0


1. 티커가 저장된 csv 파일을 불러온 후 티커를 6자리로 맞춰줍니다.
2. 빈 딕셔너리인 price_list를 생성합니다.
3. for loop 구문을 이용해 종목별 주가 데이터를 불러옵니다.
4. 종가에 해당하는 Close만을 선택한 후, 데이터프레임 형태로 만듭니다.
5. 간혹 결측치가 발생할 수 있으므로, `fillna()` 함수를 통해 결측치에는 전일 데이터를 사용합니다.

해당 작업을 통해 개별 csv 파일로 흩어져 있던 가격 데이터가 하나의 데이터로 묶이게 됩니다.

In [2]:
price_list.to_csv('data/KOR_price.csv')

마지막으로 해당 데이터를 data 폴더에 KOR_price.csv 파일로 저장합니다.

## 10.2 재무제표 정리하기

재무제표는 data/KOR_fs 폴더 내 티커_fs.csv 파일로 저장되어 있습니다. 주가는 하나의 열로 이루어져 있어 데이터를 정리하는 것이 간단했지만, 재무제표는 각 종목별 재무 항목이 모두 달라 정리하기 번거롭습니다.

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

KOR_ticker = pd.read_csv('data/KOR_ticker.csv', index_col=0)
KOR_ticker['종목코드'] = KOR_ticker['종목코드'].astype(np.str).str.zfill(6)

data_fs = {}

for i in tqdm(range(0, len(KOR_ticker))):
    name = KOR_ticker['종목코드'][i]
    data_fs[name] = pd.read_csv('data/KOR_fs/'+name+'_fs.csv', index_col=0)

100%|█████████████████████████████████████████████████████████████████████████████| 2196/2196 [00:04<00:00, 531.47it/s]


주가와 동일하게 먼저 티커 데이터를 읽어온 후, 이를 바탕으로 종목별 재무제표 데이터를 읽어와 딕셔너리에 저장합니다.

In [4]:
fs_item = data_fs['005930'].index
len(fs_item)

237

In [5]:
print(fs_item)

Index(['매출액', '매출원가', '매출총이익', '판매비와관리비', '인건비', '유무형자산상각비', '연구개발비', '광고선전비',
       '판매비', '관리비',
       ...
       '자기주식의취득', '자본구성항목의감소', '기타재무활동으로인한현금유출액', '기타재무활동으로인한현금흐름',
       '영업투자재무활동기타현금흐름', '연결범위변동으로인한현금의증가', '환율변동효과', '현금및현금성자산의증가',
       '기초현금및현금성자산', '기말현금및현금성자산'],
      dtype='object', name='IFRS(연결)', length=237)


다음으로 재무제표 항목의 기준을 정해줄 필요가 있습니다. 재무제표 작성 항목은 각 업종별로 상이하므로, 이를 모두 고려하면 지나치게 데이터가 커지게 됩니다. 또한 퀀트 투자에는 일반적이고 공통적인 항목을 주로 사용하므로 대표적인 재무 항목을 정해 이를 기준으로 데이터를 정리해도 충분합니다.

따라서 기준점으로 첫 번째 리스트, 즉 삼성전자의 재무 항목을 선택합니다. 해당 기준을 바탕으로 재무제표 데이터를 정리하며, 전체 항목에 대한 정리 이전에 간단한 예시로 첫 번째 항목인 매출액 기준 데이터 정리를 살펴보겠습니다.

In [6]:
select_fs = {}

for i in data_fs:
    # 해당 항목이 있을시 데이터를 선택
    if '매출액' in data_fs[i].index:
        select_fs[i] = data_fs[i].loc[['매출액'], :]
    # 해당 항목이 존재하지 않을 시, NA로 된 데이터프레임 생성    
    else:
        select_fs[i] = pd.DataFrame({'Blank' : [np.nan]})
        
select_fs = pd.concat(select_fs, axis=0)      

select_fs.head()

,,2017/12,2018/12,2019/12,2020/12,Blank
005930,매출액,2395754.0,2437714.0,2304009.0,2368070.0,NaN
000660,매출액,301094.0,404451.0,269907.0,319004.0,NaN
035420,매출액,46785.0,55869.0,43562.0,53041.0,NaN
051910,매출액,256980.0,281830.0,273531.0,300765.0,NaN
207940,매출액,4646.0,5358.0,7016.0,11648.0,NaN


1. select_fs의 빈 딕셔너리를 만들어 줍니다.
2. 각 종목들의 재무제표 중(data_fs) 매출액이 존재한다면 해당 행을 저장하고, 그렇지 않으면 NA로 이루어진 빈 데이터프레임을 저장합니다.
3. `concat()` 함수를 통해 행의 형태로 저장합니다.

합쳐진 데이터를 살펴보면 Blank 열이 존재하며, 이는 매출액 항목이 없는 종목의 경우 NA 데이터 프레임을 저장해 생긴 결과입니다. 이를 고려해 데이터를 클렌징합니다.

In [7]:
select_fs = select_fs.drop('Blank', axis = 1)
select_fs = select_fs.sort_index(axis = 1)
select_fs.index = KOR_ticker['종목코드']

select_fs.head()

,2017/12,2018/12,2019/12,2020/12
종목코드,,,,
005930,2395754.0,2437714.0,2304009.0,2368070.0
000660,301094.0,404451.0,269907.0,319004.0
035420,46785.0,55869.0,43562.0,53041.0
051910,256980.0,281830.0,273531.0,300765.0
207940,4646.0,5358.0,7016.0,11648.0


1. `drop()` 함수를 통해 Blank 열을 삭제합니다.
2. 열을 날짜 순서대로 정리합니다.
3. 인덱스가 티커와 재무제표 항목 두개로 이루어져 있으므로, 티커만으로 변경합니다.

해당 과정을 통해 전 종목의 매출액 데이터가 연도별로 정리되었습니다. for loop 구문을 이용해 모든 재무 항목에 대한 데이터를 정리하는 방법은 다음과 같습니다.

In [8]:
from tqdm import tqdm

# 기준이 되는 재무제표 항목
fs_item = data_fs['005930'].index

fs_list = {}

for i in tqdm(fs_item):    
    
    select_fs = {}

    for j in data_fs:        
        
        # 해당 항목이 있으면 데이터 선택
        if i in data_fs[j].index:
            select_fs[j] = data_fs[j].loc[[i], :]
        # 해당 항목 없으면 NA 데이터프레임 생성
        else:
            select_fs[j] = pd.DataFrame({'Blank' : [np.nan]})
    
    # 데이터 프레임으로 합쳐주기
    select_fs = pd.concat(select_fs, axis=0)
    
    # 빈 컬럼 삭제 후 클렌징
    select_fs = select_fs.loc[:, ~select_fs.columns.isin(['Blank'])]
    select_fs = select_fs.sort_index(axis = 1)
    select_fs.index = KOR_ticker['종목코드']
    
    fs_list[i] = select_fs

100%|████████████████████████████████████████████████████████████████████████████████| 237/237 [08:06<00:00,  2.05s/it]


위 과정을 거치면 각 재무 항목에 대한 전 종목의 연도별 데이터가 정리되어 있습니다.

In [10]:
import pickle

with open('data/KOR_fs.pickle', 'wb') as f:
    pickle.dump(fs_list, f, pickle.HIGHEST_PROTOCOL)

마지막으로 해당 데이터를 data 폴더 내에 저장합니다. 딕셔너리 형태를 그대로 저장하기 위해 피클 형태로 저장합니다.

## 10.3 가치지표 정리하기

가치지표는 data/KOR_value 폴더 내 티커_value.csv 파일로 저장되어 있습니다. 재무제표를 정리하는 방법과 거의 동일합니다.

In [9]:
import pandas as pd
import numpy as np

KOR_ticker = pd.read_csv('data/KOR_ticker.csv', index_col=0)
KOR_ticker['종목코드'] = KOR_ticker['종목코드'].astype(np.str).str.zfill(6)

In [10]:
from tqdm import tqdm

data_value = {}

for i in tqdm(range(0, len(KOR_ticker))):
    name = KOR_ticker['종목코드'][i]
    read_value = pd.read_csv('data/KOR_value/'+name+'_value.csv', index_col=0).transpose()     
        
    data_value[name] = read_value

100%|█████████████████████████████████████████████████████████████████████████████| 2196/2196 [00:08<00:00, 271.12it/s]


먼저 티커에 해당하는 파일을 불러온 후 `for loop` 구문을 통해 가치지표 데이터를 data_value 리스트에 저장합니다. 단, csv 내에 데이터가 {numref}`value_form`와 같이 행의 형태로 저장되어 있으므로, `transpose()` 함수를 이용해 형태를 바꿔줍니다.

```{table} 가치지표의 저장 예시
:name: value_form

| 밸류 | 내용 | 
| --- | --- |
| PER | Number 1 |
| PBR | Number 2 |
| PCR | Number 3 |
| PSR | Number 4 |
```

또한 가치지표가 없어 빈 데이터의 경우 NA로 채워주도록 합니다.

In [11]:
value_list = pd.concat(data_value, axis=0)

value_list.head()

,,0,PBR,PCR,PER,PSR
005930,2020/12,NaN,1.817233,7.680882,19.219868,2.117597
000660,2020/12,NaN,1.956426,8.246823,21.357349,3.183544
035420,2020/12,NaN,7.730534,44.096413,63.682596,12.031509
051910,2020/12,NaN,3.260929,10.490713,119.536000,2.037277
207940,2020/12,NaN,11.279760,256.671747,215.242158,44.534135


`concat()` 함수를 통해 딕셔너리의 데이터들을 데이터프레임 형태로 묶어줍니다. 데이터를 확인해보면 PER, PBR, PCR, PSR 열 외에 불필요한 NA로 이루어진 열이 존재합니다. 해당 열을 삭제한 후 정리 작업을 하겠습니다.

In [12]:
value_list = value_list.loc[:, ['PER', 'PBR', 'PCR', 'PSR']]
value_list.index = KOR_ticker['종목코드']

value_list.head()

,PER,PBR,PCR,PSR
종목코드,,,,
005930,19.219868,1.817233,7.680882,2.117597
000660,21.357349,1.956426,8.246823,3.183544
035420,63.682596,7.730534,44.096413,12.031509
051910,119.536000,3.260929,10.490713,2.037277
207940,215.242158,11.279760,256.671747,44.534135


1. 열 이름이 가치지표에 해당하는 부분만 선택합니다.
2. 인덱스를 티커로 변경합니다.

In [13]:
value_list.to_csv('data/KOR_value.csv')

마지막으로 data 폴더 내에 KOR_value.csv 파일로 저장합니다.